In [1]:
# Install libraries
!pip install datasets
!pip install transformers
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# wandb - Create account on and log in to https://wandb.ai/, then paste the access token
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aparnaram0 (wbnu). Use `wandb login --relogin` to force relogin


True

In [3]:
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TrainingArguments, Trainer
import torch
from torch.utils.data import random_split

In [4]:
# Load moral_stories dataset from Hugging Face - https://huggingface.co/datasets/demelin/moral_stories
dataset=load_dataset('demelin/moral_stories','full')

  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
X_train = dataset['train']['norm'][:8000]
X_test = dataset['train']['norm'][-2000:]
print("Total Dataset (Including Validation) - ", len(dataset['train']))
print("Train Dataset - ",len(X_train),'\n',"Test Dataset - ",len(X_test))

Total Dataset (Including Validation) -  12000
Train Dataset -  8000 
 Test Dataset -  2000


In [6]:
X_train

["It's responsible to keep children safe.",
 "It's wrong to become addicted to gambling.",
 "It's admirable to remain loyal to one's romantic partner.",
 "It's polite to go to a friend's wedding when invited.",
 'You should not shirk household chores.',
 "It's wrong to use violence to solve your problems.",
 "You shouldn't lie about your gender online.",
 "Family shouldn't bully other family members.",
 "You shouldn't get too demanding with your friends.",
 "It's kind to wait for people.",
 "It's rude to disrupt others with loud noises.",
 'It is bad to take from family.',
 'People are expected to keep their clothing clean.',
 'It is wrong to break the law.',
 "It's disgusting not to bathe",
 "Its wrong to be affronted when it's someone else's party.",
 'It is creepy to stand too close and stare.',
 "It's wrong not to talk to your in-laws.",
 "It's wrong to reveal one's secrets just to spite them.",
 "It's irresponsible to drink too much alcohol.",
 "It's rude to make fun of your frien

In [7]:
X_test

["It's expected that you go to your friend's birthday parties.",
 "You shouldn't delight in other people's misfortune",
 "You shouldn't talk trash about people's things.",
 "It's rude to blow up at other students.",
 "You shouldn't make your partner cry.",
 "It wrong to strangle people you don't like.",
 "You shouldn't push your partner off the bed.",
 'It is bad to cheat on your partner.',
 'It is bad to fake a death.',
 "It's good to have a relationship that lasts multiple years.",
 'It is good to tip waitresses.',
 "It's to help others when they need some assistance.",
 "It's rude to answer a phone while helping a customer.",
 "It's illegal to break into things.",
 "You shouldn't hate things.",
 'It is okay to be protective over your friends.',
 "You shouldn't fall in love with your professor.",
 "It's good to be concerned for the health of others.",
 'You should call the cops on people who steal.',
 "It's wrong to cause your child to die.",
 "You shouldn't let your kids smoke cigar

# Create GPT-2 tokenizer and define model

In [8]:
# Refer to Hugging Face documentation on Transformers https://huggingface.co/docs/transformers/v4.21.2/
'''<<Insert tokenizer, model and resize token embedding code here>>'''

tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.resize_token_embeddings(len(tokenizer))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Embedding(50259, 768)

# Tokenize data and split dataset into training and validation

In [9]:
max_length = max([len(tokenizer.encode(x)) for x in X_train])

In [10]:
class moral():
    def __init__(self, x, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for txt in x:
            #encodings_dict = tokenizer.encode(txt, max_length=max_length, pad_to_max_length = True, return_tensors='pt') 
            encodings_dict = tokenizer(txt, max_length=max_length, truncation=True, padding='max_length') 
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]


In [11]:
# Split data 80-20 to form training and validation data
dataset = moral(X_train, tokenizer, max_length=max_length)
train_size = int(0.8 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])

# Define training parameters

In [12]:
# Refer to wandb documentation to log training https://docs.wandb.ai/guides/integrations/huggingface
'''<<Insert code here for training arguments and experiment with the hyperparameters mentioned in the read me. Leave the rest to their default values.>>'''

#wandb.init(project="ai-hw7-1")
#wandb.log({"loss": loss})
#wandb.watch(model)

training_args = TrainingArguments(output_dir="test_hw7", 
                                  evaluation_strategy="epoch",
                                  learning_rate=3e-5,
                                  per_device_train_batch_size=15,
                                  per_device_eval_batch_size=15,
                                  num_train_epochs=6, 
                                  weight_decay=0.005)

In [13]:
training_args

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=every_save,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
include_inputs_for_metrics=False,
jit_mode_eval=False,
label_n

In [14]:
%env WANDB_WATCH=all
%env WANDB_SILENT=true

env: WANDB_WATCH=all
env: WANDB_SILENT=true


# Train model

In [15]:
'''<<Insert code for trainer here>>'''

device = torch.device(torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))
model = model.to(device)

with wandb.init(project="ai-hw7") as run:

  trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset, eval_dataset=val_dataset, data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                              'attention_mask': torch.stack([f[1] for f in data]),
                                                              'labels': torch.stack([f[0] for f in data])})

  trainer.train()  

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 6400
  Num Epochs = 6
  Instantaneous batch size per device = 15
  Total train batch size (w. parallel, distributed & accumulation) = 15
  Gradient Accumulation steps = 1
  Total optimization steps = 2562
  Number of trainable parameters = 124441344
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss
1,No log,1.472115
2,2.667700,1.444854
3,1.444600,1.429698
4,1.341700,1.432214
5,1.282600,1.438356
6,1.249900,1.442498


***** Running Evaluation *****
  Num examples = 1600
  Batch size = 15
Saving model checkpoint to test_hw7/checkpoint-500
Configuration saved in test_hw7/checkpoint-500/config.json
Model weights saved in test_hw7/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1600
  Batch size = 15
Saving model checkpoint to test_hw7/checkpoint-1000
Configuration saved in test_hw7/checkpoint-1000/config.json
Model weights saved in test_hw7/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1600
  Batch size = 15
Saving model checkpoint to test_hw7/checkpoint-1500
Configuration saved in test_hw7/checkpoint-1500/config.json
Model weights saved in test_hw7/checkpoint-1500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1600
  Batch size = 15
Saving model checkpoint to test_hw7/checkpoint-2000
Configuration saved in test_hw7/checkpoint-2000/config.json
Model weights saved in test_hw7/checkpoint-2000/pytorch_model.bin
***** R

eval/loss,█▄▁▁▂▃
eval/runtime,█▇▇▁▁▁
eval/samples_per_second,▁▂▂███
eval/steps_per_second,▁▂▂███
train/epoch,▁▁▂▃▄▅▅▆▇███
train/global_step,▁▁▂▃▄▅▅▆▇███
train/learning_rate,█▆▄▃▁
train/loss,█▂▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


In [16]:
wandb.finish()

# Inference using sample data

In [17]:
'''<<Insert code for Tokenize sample data and use model.generate function to get predicted logits and decode them using tokenizer.decode function refer hugging face documentation for further information >>'''

prompt='Parents are allowed to decide'

device = torch.device("cpu")
model = model.to(device)

test_input_ids = tokenizer(prompt, return_tensors='pt').input_ids
test_output = model.generate(test_input_ids, max_new_tokens=9, do_sample=True, top_k=50, top_p=0.95)

output = tokenizer.batch_decode(test_output, skip_special_tokens=True)
print(output)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Parents are allowed to decide who is best for their children.']


# Inference using test dataset

In [18]:
'''<<Insert code for infering every sentence in test dataset X_test >>'''

from nltk.translate.bleu_score import sentence_bleu
from tqdm import tqdm

bleu_scores = []
hypotheses = []

device = torch.device("cpu")
model = model.to(device)

for reference in X_test:
  try:
    first, second = reference[:len(reference)//2], reference[len(reference)//2:]

    test_input_ids = tokenizer(first, return_tensors='pt').input_ids
    test_output = model.generate(test_input_ids, max_new_tokens=9, do_sample=True, top_k=50, top_p=0.95)
    hypothesis = tokenizer.batch_decode(test_output, skip_special_tokens=True)
    print(reference)
    print(hypothesis)

    bleu_score = sentence_bleu([reference], hypothesis[0])
    print(bleu_score)

    bleu_scores.append(bleu_score)
    hypotheses.append(hypothesis)

  except Exception as e:
    continue




The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


It's expected that you go to your friend's birthday parties.
["It's expected that you go to y's events."]
0.49935609661943214


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


You shouldn't delight in other people's misfortune
["You shouldn't delight in  people's urinals."]
0.6760734879515312


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

You must try to be faithful to your significant other.
['You must try to be faithful to your spouse']
0.6673519644895678


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


It's bad to get enraged at people.
["It's bad to get e-mails from your friends."]
0.43522973489251776


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


It's wrong to snap at your friends.
["It's wrong to snaat to date someone with bipolar disorder."]
0.3400748443581816
It's bad when a father is often absent.
["It's bad when a fat person sneaks into your house."]
0.4587505082857642


# Evaluate model using BLEU score

In [19]:
# Compare predicted sentences from test data with actual test data. Refer to NLTK documentation for sentence_bleu
import statistics
from nltk.translate.bleu_score import sentence_bleu

average_bleu = sum(bleu_scores) / len(bleu_scores)
print("Average bleu score: ", average_bleu)

Average bleu score:  0.5142531362264792
